<a href="https://colab.research.google.com/github/RevSLightfoot/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
# Importing all of SciPy to play with other options (time permitting)
import scipy.stats as scipy

In [2]:
# Dataset to be used
#  - https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2019-10-07 20:00:43--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2019-10-07 20:00:43 (284 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
df =

In [6]:
df = pd.read_csv('house-votes-84.data', header=None)
print(df.shape)
df.head()

(435, 17)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [8]:
columns= ['party','handicapped-infants','water-project','budget',
          'physician-fee-freeze', 'el-salvador-aid','religious-groups',
          'anti-satellite-ban','aid-to-contras','mx-missile','immigration',
          'synfuels', 'education', 'right-to-sue','crime','duty-free',
          'south-africa']

df.columns = columns
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [13]:
for col in df.columns:
  print(df[col].value_counts())

democrat      267
republican    168
Name: party, dtype: int64
n    236
y    187
?     12
Name: handicapped-infants, dtype: int64
y    195
n    192
?     48
Name: water-project, dtype: int64
y    253
n    171
?     11
Name: budget, dtype: int64
n    247
y    177
?     11
Name: physician-fee-freeze, dtype: int64
y    212
n    208
?     15
Name: el-salvador-aid, dtype: int64
y    272
n    152
?     11
Name: religious-groups, dtype: int64
y    239
n    182
?     14
Name: anti-satellite-ban, dtype: int64
y    242
n    178
?     15
Name: aid-to-contras, dtype: int64
y    207
n    206
?     22
Name: mx-missile, dtype: int64
y    216
n    212
?      7
Name: immigration, dtype: int64
n    264
y    150
?     21
Name: synfuels, dtype: int64
n    233
y    171
?     31
Name: education, dtype: int64
y    209
n    201
?     25
Name: right-to-sue, dtype: int64
y    248
n    170
?     17
Name: crime, dtype: int64
n    233
y    174
?     28
Name: duty-free, dtype: int64
y    269
?    104
n     62
Name: 

In [14]:
convertToBool = {'?':np.NaN, 'n':0, 'y':1}

df = df.replace(convertToBool)

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [20]:
df.isnull().sum()

party                     0
handicapped-infants      12
water-project            48
budget                   11
physician-fee-freeze     11
el-salvador-aid          15
religious-groups         11
anti-satellite-ban       14
aid-to-contras           15
mx-missile               22
immigration               7
synfuels                 21
education                31
right-to-sue             25
crime                    17
duty-free                28
south-africa            104
dtype: int64

In [24]:
rep = df[df.party == "republican"]
print("Rep:", rep.shape)

dem = df[df.party == "democrat"]
print("Dem:", dem.shape)



Rep: (168, 17)
Dem: (267, 17)


In [0]:
from IPython.display import display
import ipywidgets as widgets

In [0]:
subject_list = columns[1:]
subject = widgets.Dropdown(options=subject_list)

party_list = ['rep', 'dem']
party = widgets.Dropdown(options=party_list)

tol = widgets.FloatSlider(
    value=0,
    min=0,
    max=1.0,
    step=0.25,
    description='tolerance',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [119]:
button = widgets.Button(description="Update!")
output = widgets.interactive_output(ttests_out, {'subject': subject, 'tol': tol})

def ttests_out(subject, tol):
  with output:
    output.clear_output()
    print('\n Republican:\n', scipy.ttest_1samp(rep[subject],
                                                tol,
                                                nan_policy='omit'))
    print('\n Democrat:\n', scipy.ttest_1samp(dem[subject],
                                                tol,
                                                nan_policy='omit'))
    print('\n', scipy.ttest_ind(rep[subject],
                                dem[subject],
                                nan_policy='omit'))

widgets.VBox([widgets.HBox([subject, tol]), output])
    
# button.on_click(on_button_clicked)